# 21 Feb 2023
# nrobot

https://online.drl.wi.gov/orders/searchorders.aspx
 - Wisconsin Department of Safety and Professional Services
 - Health & Business Renewal Application 

In [2]:
# https://towardsdatascience.com/how-to-extract-text-from-pdf-245482a96de7

# install libraries

In [3]:
#!pip install pdfplumber
# !pip install pdfminer3 # old / unmaintained
# https://github.com/pdfminer/pdfminer.six
# !pip install pdfminer.six # also fails -- I realized that's because the 
# text of the order is actually scanned (image?)
#!pip install borb
# https://pyimagesearch.com/2021/08/16/installing-tesseract-pytesseract-and-python-ocr-packages-on-your-system/
#!pip install pytesseract
#!brew install freetype imagemagick
#!pip install pdf2image

# try various things

In [3]:
# only extracts the first (easy) page 
import pdfplumber 
with pdfplumber.open(r'./ORDER0008271-00019637.pdf') as pdf:
    first_page = pdf.pages[0]
    #print(first_page.extract_text())

In [4]:
from pdfminer.high_level import extract_text

text = extract_text("./ORDER0008271-00019637.pdf")
#print(text)

In [5]:
# pdf info: created with www.pdfsharp.com

In [6]:
# https://stackabuse.com/applying-ocr-to-a-scanned-pdf-in-python-using-borb/

In [7]:
# New imports
import typing
from pathlib import Path

from borb.pdf import Document
from borb.pdf import Page
from borb.toolkit import SimpleTextExtraction
#from borb.pdf import SingleColumnLayout
#from borb.pdf import Paragraph
from borb.pdf import PDF

def read_modified_document():

    # read the Document
    doc: typing.Optional[Document] = None
    l: SimpleTextExtraction = SimpleTextExtraction()

    with open("./ORDER0008271-00019637.pdf", "rb") as in_file_handle:
        #doc = PDF.loads(in_file_handle)
        doc = PDF.loads(in_file_handle, [l])

    # check whether we have read a Document
    assert doc is not None

    # print the \Producer key from the \Info dictionary
    print("Producer: %s" % doc.get_document_info().get_producer())
    print(f'Num. of pages: {len(l.get_text())}')
    print(l.get_text()[0][:500])
    #print(f'# of Pages: {l.shape}')
    return l

l = read_modified_document()


Producer: PDFsharp 1.31.1789-g \(www.pdfsharp.com\)
Num. of pages: 1
WISCONSIN DEPARTMENT OF
SAFETY AND PROFESSIONAL SERVICES
Wisconsin Department of Safety and Professional Services
Access to the Public Records of the Reports of Decisions
This Reports of Decisions document was retrieved from the Wisconsin Department of Safety and
Professional Services website. These records are open to public view under Wisconsin’s Open
Records law, sections 19.31-19.39 Wisconsin Statutes.
Please read this agreement prior to viewing the Decision:
 The Reports of Decisions is de


In [8]:
# https://github.com/jorisschellekens/borb-examples#72-performing-ocr-on-a-pdf

In [9]:
!brew info tesseract

==> tesseract: stable 5.3.0 (bottled), HEAD
OCR (Optical Character Recognition) engine
https://github.com/tesseract-ocr/
/opt/homebrew/Cellar/tesseract/5.3.0_1 (73 files, 32.2MB) *
  Poured from bottle on 2023-01-23 at 18:01:00
From: https://github.com/Homebrew/homebrew-core/blob/HEAD/Formula/tesseract.rb
License: Apache-2.0
==> Dependencies
Build: autoconf ✔, automake ✔, libtool ✔, pkg-config ✔
Required: cairo ✔, icu4c ✔, leptonica ✔, libarchive ✔, pango ✔
==> Options
--HEAD
	Install HEAD version
==> Caveats
This formula contains only the "eng", "osd", and "snum" language data files.
If you need any other supported languages, run `brew install tesseract-lang`.
==> Analytics
install: 175,981 (30 days), 465,732 (90 days), 1,465,418 (365 days)
install-on-request: 66,153 (30 days), 146,394 (90 days), 345,527 (365 days)
build-error: 10 (30 days)


In [10]:
# TesseractError: (1, 'Error opening data file /opt/homebrew/Cellar/tesseract/5.3.0_1/share/eng.traineddata Please make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory. Failed loading language \'eng\' Tesseract couldn\'t load any languages! Could not initialize tesseract.')

# > Fix with brew info tesseract + share + tessdata, as below

In [11]:
!ls /opt/homebrew/Cellar/tesseract/5.3.0_1/share/tessdata

configs          osd.traineddata  snum.traineddata
eng.traineddata  pdf.ttf          tessconfigs


In [12]:
from borb.pdf import Document
from borb.pdf import PDF
from borb.toolkit.ocr.ocr_as_optional_content_group import OCRAsOptionalContentGroup

from pathlib import Path

# set up everything for OCR
tesseract_data_dir: Path = Path("/opt/homebrew/Cellar/tesseract/5.3.0_1/share/tessdata")
assert tesseract_data_dir.exists()
#l: OCRAsOptionalContentGroup = OCRAsOptionalContentGroup(tesseract_data_dir)
l: SimpleTextExtraction = SimpleTextExtraction()

# read Document
doc: typing.Optional[Document] = None
#with open("ORDER0008271-00019637.pdf", "rb") as pdf_file_handle:
with open("page3.pdf", "rb") as pdf_file_handle:
    doc = PDF.loads(pdf_file_handle, [l])

assert doc is not None
len(l.get_text())
l.get_text()


{}

In [ ]:
# https://stackoverflow.com/questions/45480280/convert-scanned-pdf-to-text-python

# try Imagemagick OCR

In [ ]:
# ImportError: MagickWand shared library not found.
# You probably had not installed ImageMagick library.
# cannot find libwand, libmagick
# https://stackoverflow.com/a/71591742/851192 

In [ ]:
!brew info imagemagick


==> imagemagick: stable 7.1.0-62 (bottled), HEAD
Tools and libraries to manipulate images in many formats
https://imagemagick.org/index.php
/opt/homebrew/Cellar/imagemagick/7.1.0-62 (807 files, 31MB) *
  Poured from bottle using the formulae.brew.sh API on 2023-02-21 at 13:53:45
From: https://github.com/Homebrew/homebrew-core/blob/HEAD/Formula/imagemagick.rb
License: ImageMagick
==> Dependencies
Build: pkg-config ✔
Required: freetype ✔, ghostscript ✔, jpeg-turbo ✔, libheif ✔, liblqr ✔, libpng ✔, libraw ✔, libtiff ✔, libtool ✔, little-cms2 ✔, openexr ✔, openjpeg ✔, webp ✔, xz ✔, libomp ✔
==> Options
--HEAD
	Install HEAD version
==> Analytics
install: 214,522 (30 days), 517,268 (90 days), 2,006,493 (365 days)
install-on-request: 188,777 (30 days), 458,143 (90 days), 1,777,905 (365 days)
build-error: 26 (30 days)


In [12]:
#!echo 'export MAGICK_HOME=/opt/homebrew/Cellar/imagemagick/7.1.0-62/' >> ~/.bash_profile
#!echo 'export PATH="$MAGICK_HOME/bin:$PATH"' >> ~/.bash_profile
#!echo 'export PATH="/opt/homebrew/Cellar/imagemagick/7.1.0-62/bin:$PATH"' >> ~/.bash_profile
!echo $PATH | grep brew


/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/

##  !--- current

In [2]:
!echo $PATH


/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/v3/bin:/Users/owls/

In [5]:
import wand.version
print(wand.version.MAGICK_VERSION)

AttributeError: module 'wand.version' has no attribute 'MAGICK_VERSION'

In [3]:
PATH="$MAGICK_HOME/bin:$PATH"
PATH="/opt/homebrew/Cellar/imagemagick/7.1.0-62/bin:$PATH"

In [4]:
import os
import io
from PIL import Image
import pytesseract
from wand.image import Image as wi
import gc

pdf_path = './page3.pdf'

pdf=wi(filename=pdf_path,resolution=300)
pdfImg=pdf.convert('jpeg')

imgBlobs=[]
extracted_text=[]

def Get_text_from_image(pdf_path):
    pdf=wi(filename=pdf_path,resolution=300)
    pdfImg=pdf.convert('jpeg')
    imgBlobs=[]
    extracted_text=[]
    for img in pdfImg.sequence:
        page=wi(image=img)
        imgBlobs.append(page.make_blob('jpeg'))

    for imgBlob in imgBlobs:
        im=Image.open(io.BytesIO(imgBlob))
        text=pytesseract.image_to_string(im,lang='eng')
        extracted_text.append(text)

    return (extracted_text)

ImportError: MagickWand shared library not found.
You probably had not installed ImageMagick library.
Try to install:
  brew install freetype imagemagick

In [ ]:
# https://stackoverflow.com/questions/45480280/convert-scanned-pdf-to-text-python
import glob
from pdf2image import convert_from_path

pdfs = glob.glob(r"page3.pdf")

for pdf_path in pdfs:
    pages = convert_from_path(pdf_path, 500)

    for pageNum,imgBlob in enumerate(pages):
        text = pytesseract.image_to_string(imgBlob,lang='eng')

        with open(f'{pdf_path[:-4]}_page{pageNum}.txt', 'w') as the_file:
            the_file.write(text)